In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utils, Networks

In [3]:
import numpy as np
import pandas as pd
import random
import os
import time
from math import ceil
from sklearn.preprocessing import StandardScaler
from utils import load_embeddings_and_ids, concatenate_featmats, User, VisualSimilarityHandler, get_decaying_learning_rates

In [4]:
# use a single GPU because we want to be nice with other people :)
os.environ["CUDA_VISIBLE_DEVICES"]="0"

###  Load pre-trained image embeddings

In [5]:
resnet50 = load_embeddings_and_ids('/mnt/workspace/Ugallery/ResNet50/', 'flatten_1.npy', 'ids')

In [6]:
# alexnet = load_embeddings_and_ids('/mnt/workspace/Ugallery/AlexNet/', 'fc7.npy', 'ids.npy')

In [7]:
# inceptionv3 = load_embeddings_and_ids('/mnt/workspace/Ugallery/InceptionV3/', 'avg_pool.npy', 'ids')

###  Concatenate embeddings + z-score normalization

In [8]:
embedding_list = [
    resnet50,
#     alexnet,
#     inceptionv3,
]

In [9]:
artwork_ids_set = set()
for embedding in embedding_list:
    artwork_ids_set.update(embedding['index2id'])
artwork_ids = list(artwork_ids_set)
artwork_id2index = {_id:i for i,_id in enumerate(artwork_ids)}
n_artworks = len(artwork_ids)
n_artworks

13297

In [10]:
featmat_list = [tmp['featmat'] for tmp in embedding_list]
id2index_list = [tmp['id2index'] for tmp in embedding_list]
concat_featmat = concatenate_featmats(artwork_ids, featmat_list, id2index_list)

In [11]:
concat_featmat = StandardScaler().fit_transform(concat_featmat)

In [ ]:
concat_featmat[0], concat_featmat[]

In [12]:
concat_featmat.shape

(13297, 2048)

###  Load clusters

In [13]:
import json

In [14]:
# with open('/mnt/workspace/Ugallery/Clustering/artworkId2clusterId(resnet50+alexnet+inceptionv3+vgg19+incepresv2).json') as f:
with open('/mnt/workspace/Ugallery/Clustering/artworkId2clusterId(resnet50).json') as f:
    artworkId2clusterId = json.load(f)
cluster_ids = np.full((n_artworks,), -1, dtype=int)
for k, v in artworkId2clusterId.items():
    cluster_ids[artwork_id2index[int(k)]] = v

In [15]:
cluster_ids.min(), cluster_ids.max(), cluster_ids.shape

(0, 99, (13297,))

In [16]:
n_clusters = len(set(cluster_ids))
n_clusters

100

In [17]:
clusterId2artworkIndexes = [[] for _ in range(n_clusters)]
for i, cluster_id in enumerate(cluster_ids):
    clusterId2artworkIndexes[cluster_id].append(i)

###  Load PCA200 embeddings

In [18]:
pca200 = load_embeddings_and_ids(
#     '/mnt/workspace/Ugallery/PCA200(resnet50+alexnet+inceptionv3+vgg19+incepresv2)/',
    '/mnt/workspace/Ugallery/PCA200(resnet50)/',
    'embeddings.npy',
    'ids.npy',
)

In [19]:
pca200_embeddings = pca200['featmat']
pca200_index2id = pca200['index2id']
pca200_id2index = pca200['id2index']

In [20]:
pca200_embeddings.shape

(13297, 200)

In [21]:
assert np.array_equal(artwork_ids, pca200_index2id)

###  Load transactions

In [22]:
sales_df = pd.read_csv('./data/valid_sales.csv')
artworks_df = pd.read_csv('./data/valid_artworks.csv')

In [23]:
artist_ids = np.full((n_artworks,), -1, dtype=int)
for _artworkId, _artistId in zip(artworks_df.id, artworks_df.artist_id):
    i = artwork_id2index[_artworkId]
    artist_ids[i] = _artistId

In [24]:
artistId2artworkIndexes = dict()
for i, _artistId in enumerate(artist_ids):
    if _artistId == -1:
        continue
    try:
        artistId2artworkIndexes[_artistId].append(i)
    except KeyError:
        artistId2artworkIndexes[_artistId] = [i]

### Collect transactions per user (making sure we hide the last nonfirst purchase basket per user)

#### create list of users

In [25]:
user_ids = sales_df.customer_id.unique()
user_id2index = { _id:i for i,_id in enumerate(user_ids) }
users = [User(uid) for uid in user_ids]
n_users = len(user_ids)
n_users

2919

#### collect and sanity check transactions per user

In [26]:
sorted_sales_df = sales_df.sort_values('order_date')

In [27]:
# clear structures to prevent possible duplicate elements
for user in users:
    user.clear()

# collect transactions per user sorted by timestamp
for uid, aid, t in zip(sorted_sales_df.customer_id,
                       sorted_sales_df.artwork_id,
                       sorted_sales_df.order_date):
    users[user_id2index[uid]].append_transaction(
        aid, t, artwork_id2index, artist_ids, cluster_ids)
    assert users[user_id2index[uid]]._uid == uid
    
# bin transctions with same timestamps into purchase baskets
for user in users:
    user.build_purchase_baskets()
    user.sanity_check_purchase_baskets()
    user.remove_last_nonfirst_purchase_basket(
        artwork_id2index, artist_ids, cluster_ids)
    user.sanity_check_purchase_baskets()
    user.refresh_nonpurchased_cluster_ids(n_clusters)
    user.refresh_cluster_ids()
    user.refresh_artist_ids()

### Generate training data

In [28]:
def hash_triple(ui, pi, ni):
    return  ((pi * n_artworks) + ni) * n_users + ui

In [29]:
def sanity_check_instance(instance,
                          pos_in_profile=True,
                          pos_sharing_cluster_artist=None,
                          neg_notsharing_artist=None,
                          neg_notsharing_artist_cluster=None,
                         ):
    ui, pi, ni = instance    
    try:
        assert 0 <= ui < n_users
        assert 0 <= pi < n_artworks
        assert 0 <= ni < n_artworks
        assert pi != ni
        assert not vissimhandler.same(pi,ni)
        
        assert 0 <= ui < n_users
        user = users[ui]
        
        if pos_in_profile is True:
            assert pi in user.artwork_idxs_set
        elif pos_in_profile is False:
            assert pi not in user.artwork_idxs_set
            
        if pos_sharing_cluster_artist:
            assert artist_ids[pi] in user.artist_ids_set
            assert cluster_ids[pi] in user.cluster_ids_set
            
        assert ni not in user.artwork_idxs_set
        if neg_notsharing_artist:
            assert artist_ids[ni] not in user.artist_ids_set
        if neg_notsharing_artist_cluster:
            assert artist_ids[ni] not in user.artist_ids_set
            assert cluster_ids[ni] not in user.cluster_ids_set            
    except AssertionError:
        print('ui = ', ui)
        print('pi = ', pi)
        print('ni = ', ni)
        raise

In [30]:
def append_instance(container, instance, **kwargs):
    global _hash_collisions
    ui, pi, ni = instance
    
    h = hash_triple(ui, pi, ni)
    if h in used_hashes:
        _hash_collisions += 1
        return False
    
    if vissimhandler.same(pi, ni):
        return False
    
    sanity_check_instance(instance, **kwargs)
    container.append(instance)
    used_hashes.add(h)
    return True

In [31]:
def print_num_samples(sampler_func):
    def wrapper(instances_container, n_samples):
        len_before = len(instances_container)
        sampler_func(instances_container, n_samples)
        actual_samples = len(instances_container) - len_before
        print('  target samples: %d' % n_samples)
        print('  actual samples: %d' % actual_samples)
        print('  delta: %d' % (n_samples - actual_samples))
    return wrapper

In [32]:
vissimhandler = VisualSimilarityHandler(cluster_ids, pca200_embeddings)

In [33]:
vissimhandler.count = 0
used_hashes = set()
_hash_collisions = 0
train_instances = []
test_instances = []

In [34]:
N_STRATEGIES = 1
TOTAL_SAMPLES__TRAIN = 10000000
TOTAL_SAMPLES__TEST =  400000
N_SAMPLES_PER_STRATEGY__TRAIN = int(TOTAL_SAMPLES__TRAIN / N_STRATEGIES)
N_SAMPLES_PER_STRATEGY__TEST = int(TOTAL_SAMPLES__TEST / N_STRATEGIES)
N_SAMPLES_PER_STRATEGY__TRAIN, N_SAMPLES_PER_STRATEGY__TEST

(10000000, 400000)

In [35]:
FINE_GRAINED_THRESHOLD = 0.6
VISUAL_CONFIDENCE_THRESHOLD = 0.1

## Original BPR strategy

#### 1) given profile, recommend profile
Given a user's profile, all items in the profile should be ranked higher than items outside the profile

In [36]:
def sample_artwork_index__outsideprofile(profile_set, pi):
    while True:
        if random.random() <= FINE_GRAINED_THRESHOLD:
            ni = random.choice(clusterId2artworkIndexes[cluster_ids[pi]])
        else:            
            c = random.randint(0, n_clusters-1)
            ni = random.choice(clusterId2artworkIndexes[c])
        if ni not in profile_set:
            return ni

In [37]:
@print_num_samples
def generate_samples__rank_profile_above_nonprofile(instances_container, n_samples):
    n_samples_per_user = ceil(n_samples / n_users)    
    for ui, user in enumerate(users):
        profile = user.artwork_idxs
        profile_set = user.artwork_idxs_set
        for _ in range(n_samples_per_user):
            for __ in range(5):
                pi = random.choice(profile)
                ni = sample_artwork_index__outsideprofile(profile_set, pi)
                if append_instance(instances_container, (ui, pi, ni)):
                    break

In [42]:
print('sampling train instances ...')
generate_samples__rank_profile_above_nonprofile(
#     train_instances, n_samples=N_SAMPLES_PER_STRATEGY__TRAIN)
    train_instances, n_samples=4419)
# print('sampling test instances ...')
# generate_samples__rank_profile_above_nonprofile(
# #     test_instances, n_samples=N_SAMPLES_PER_STRATEGY__TEST)
#     test_instances, n_samples=212)
print(len(train_instances), len(test_instances))
print('hash_collisions = ', _hash_collisions)
print('visual_collisions = ', vissimhandler.count)

sampling train instances ...
  target samples: 4419
  actual samples: 4695
  delta: -276
10000276 402101
hash_collisions =  18146607
visual_collisions =  491


## Domain-specific strategies

##### 2) Recommend visually similar items from favorite artists
Given a user, any item outside the user's profile that shares artist and visual cluster with items in the user's profile should be ranked higher than any item from an artist and visual cluster not present in the user's profile

In [47]:
def sample_artwork_index__outsideprofile__sharing_artist_cluster(profile_set, artists_list, clusters_set):
    for _ in range(20): # try at most 20 times
        # sharing artist
        a = random.choice(artists_list)
        i = random.choice(artistId2artworkIndexes[a])
        # sharing cluster
        if cluster_ids[i] not in clusters_set: continue
        # oustide profile
        if i in profile_set: continue
        # done
        return i
    return None # failed to find

In [48]:
def sample_artwork_index__notsharing_artist_cluster(artists_set, nonused_clusters_list):
    while True:
        # not sharing cluster
        c = random.choice(nonused_clusters_list)
        i = random.choice(clusterId2artworkIndexes[c])
        # not sharing artist
        if artist_ids[i] not in artists_set:
            return i

In [49]:
@print_num_samples
def generate_samples__rank_sharing_artist_cluster_above_notsharing_artist_cluster(instances_container, n_samples):
    n_samples_per_user = ceil(n_samples / n_users)    
    for ui, user in enumerate(users):
        profile = user.artwork_idxs
        profile_set = user.artwork_idxs_set
        artists_list = user.artist_ids
        artists_set = user.artist_ids_set
        clusters_set = user.cluster_ids_set
        nonused_clusters_list = user.nonp_cluster_ids
        for _ in range(n_samples_per_user):
            for __ in range(5):
                pi = sample_artwork_index__outsideprofile__sharing_artist_cluster(profile_set, artists_list, clusters_set)
                if pi is None: continue
                ni = sample_artwork_index__notsharing_artist_cluster(artists_set, nonused_clusters_list)
                if append_instance(instances_container, (ui, pi, ni),
                                   pos_in_profile=False,
                                   pos_sharing_cluster_artist=True,
                                   neg_notsharing_artist_cluster=True,
                                  ):
                    break

In [55]:
print('sampling train instances ...')
generate_samples__rank_sharing_artist_cluster_above_notsharing_artist_cluster(
#     train_instances, n_samples=N_SAMPLES_PER_STRATEGY__TRAIN)
    train_instances, n_samples=3908)
# print('sampling test instances ...')
# generate_samples__rank_sharing_artist_cluster_above_notsharing_artist_cluster(
# #     test_instances, n_samples=N_SAMPLES_PER_STRATEGY__TEST)
#     test_instances, n_samples=703)
print(len(train_instances), len(test_instances))
print('hash_collisions = ', _hash_collisions)
print('visual_collisions = ', vissimhandler.count)

sampling train instances ...
  target samples: 3908
  actual samples: 4372
  delta: -464
10001489 403310
hash_collisions =  7004532
visual_collisions =  228


#### shuffle train instances

In [43]:
random.shuffle(train_instances)

### Training Model

In [44]:
def generate_minibatches(tuples, batch_size):
    n_tuples = len(tuples)
    n_batches = ceil(n_tuples / batch_size)
    
    assert n_batches * batch_size >= n_tuples
    assert (n_batches - 1) * batch_size < n_tuples
    
    indexes = list(range(n_tuples))
    random.shuffle(indexes)
    
    print('n_tuples = ', n_tuples)
    print('n_batches = ', n_batches)
    
    user_index_batches = [None] * n_batches
    pos_index_batches = [None] * n_batches
    neg_index_batches = [None] * n_batches
    
    for i in range(n_batches):
        jmin = i * batch_size
        jmax = min(jmin + batch_size, n_tuples)
        actual_batch_size = jmax - jmin
        
        user_index_batch = np.empty((actual_batch_size,), dtype=int)
        pos_index_batch = np.empty((actual_batch_size,), dtype=int)
        neg_index_batch = np.empty((actual_batch_size,), dtype=int)
        
        for j in range(actual_batch_size):
            t = tuples[indexes[jmin+j]]
            user_index_batch[j] = t[0]
            pos_index_batch[j] = t[1]
            neg_index_batch[j] = t[2]

        user_index_batches[i] = user_index_batch
        pos_index_batches[i] = pos_index_batch
        neg_index_batches[i] = neg_index_batch
        
    return dict(
        user_index_batches = user_index_batches,
        pos_index_batches  = pos_index_batches,
        neg_index_batches  = neg_index_batches,
        n_batches          = n_batches,
    )

In [45]:
def sanity_check_minibatches(minibatches):
    user_index_batches = minibatches['user_index_batches']
    pos_index_batches = minibatches['pos_index_batches']
    neg_index_batches = minibatches['neg_index_batches']
    n_batches = minibatches['n_batches']
    assert n_batches == len(user_index_batches)
    assert n_batches == len(pos_index_batches)
    assert n_batches == len(neg_index_batches)
    assert n_batches > 0
    
    for user_index, pos_index, neg_index in zip(
        user_index_batches,
        pos_index_batches,
        neg_index_batches
    ):
        n = user_index.shape[0]
        assert n == pos_index.shape[0]
        assert n == neg_index.shape[0]
        
        for i in range(n):
            ui = user_index[i]
            pi = pos_index[i]
            ni = neg_index[i]
            assert pi != ni
            assert ni not in users[ui].artwork_idxs_set

In [46]:
import tensorflow as tf
from Networks import VBPR_Network_Train, TrainLogger

In [47]:
def train_network(train_minibatches, test_minibatches,
                  n_train_instances, n_test_instances, batch_size,
                  pretrained_embeddings,
                  user_latent_dim,
                  item_latent_dim,
                  item_visual_dim,
                  model_path,
                  max_seconds_training=3600,
                  min_seconds_to_check_improvement=60,
                  early_stopping_checks=4,
                  weight_decay=0.001,
                  learning_rates=[1e-3]):
    
    n_train_batches = train_minibatches['n_batches']
    n_test_batches = test_minibatches['n_batches']
    
    print('learning_rates = ', learning_rates)
    
    with tf.Graph().as_default():
        network = VBPR_Network_Train(
            n_users=n_users,
            n_items=n_artworks,
            user_latent_dim=user_latent_dim,
            item_latent_dim=item_latent_dim,
            item_visual_dim=item_visual_dim,            
            pretrained_dim=pretrained_embeddings.shape[1],
            weight_decay=weight_decay,
        )
        
        print('Variables to be trained:')
        for x in tf.global_variables():
            print('\t', x)
        
        with tf.Session() as sess:
            try:
                saver = tf.train.Saver()            
                saver.restore(sess, tf.train.latest_checkpoint(model_path))
                print('model successfully restored from checkpoint!')
            except ValueError:
                print('no checkpoint found: initializing variables with random values')
                os.makedirs(MODEL_PATH, exist_ok=True)
                sess.run(tf.global_variables_initializer())            
            trainlogger = TrainLogger(model_path + 'train_logs.csv')

            # ========= BEFORE TRAINING ============
            
            initial_test_acc = 0.            
            for user_index, pos_index, neg_index in zip(
                test_minibatches['user_index_batches'],
                test_minibatches['pos_index_batches'],
                test_minibatches['neg_index_batches']
            ):
                minibatch_test_acc = network.get_test_accuracy(
                    sess, pretrained_embeddings, user_index, pos_index, neg_index)
                initial_test_acc += minibatch_test_acc
            initial_test_acc /= n_test_instances

            print("Before training: test_accuracy = %f" % initial_test_acc)
            
            best_test_acc = initial_test_acc
            seconds_training = 0
            elapsed_seconds_from_last_check = 0
            checks_with_no_improvement = 0
            last_improvement_loss = None
            
            # ========= TRAINING ============
            
            print ('Starting training ...')
            n_lr = len(learning_rates)
            lr_i = 0
            train_loss_ema = None # exponential moving average
            
            while seconds_training < max_seconds_training:
                
                for train_i, (user_index, pos_index, neg_index) in enumerate(zip(
                    train_minibatches['user_index_batches'],
                    train_minibatches['pos_index_batches'],
                    train_minibatches['neg_index_batches']
                )):
                    # optimize and get traing loss
                    start_t = time.time()
                    _, minibatch_train_loss = network.optimize_and_get_train_loss(
                        sess, pretrained_embeddings, user_index, pos_index, neg_index, learning_rates[lr_i])
                    delta_t = time.time() - start_t
                    
                    # update train loss exponential moving average
                    train_loss_ema = minibatch_train_loss if train_loss_ema is None else\
                                    0.999 * train_loss_ema + 0.001 * minibatch_train_loss
                    
                    # update time tracking variables
                    seconds_training += delta_t
                    elapsed_seconds_from_last_check += delta_t
                    
                    # check for improvements using test set if it's time to do so
                    if elapsed_seconds_from_last_check >= min_seconds_to_check_improvement:
                        
                        # --- testing                        
                        test_acc = 0.
                        for _user_index, _pos_index, _neg_index in zip(
                            test_minibatches['user_index_batches'],
                            test_minibatches['pos_index_batches'],
                            test_minibatches['neg_index_batches']
                        ):
                            minibatch_test_acc = network.get_test_accuracy(
                                sess, pretrained_embeddings, _user_index, _pos_index, _neg_index)
                            test_acc += minibatch_test_acc
                        test_acc /= n_test_instances
                    
                        print(("train_i=%d, train_loss = %.12f, test_accuracy = %.6f,"
                               " check_secs = %.2f, total_secs = %.2f") % (
                                train_i, train_loss_ema, test_acc, elapsed_seconds_from_last_check, seconds_training))                        
                        
                        # check for improvements
                        if (test_acc > best_test_acc) or (
                            test_acc == best_test_acc and (
                                last_improvement_loss is not None and\
                                last_improvement_loss > train_loss_ema
                            )
                        ):  
                            last_improvement_loss = train_loss_ema
                            best_test_acc = test_acc
                            checks_with_no_improvement = 0
                            saver = tf.train.Saver()
                            save_path = saver.save(sess, MODEL_PATH)                    
                            print("   ** improvement detected: model saved to path ", save_path)
                            model_updated = True
                        else:
                            checks_with_no_improvement += 1                            
                            model_updated = False

                        # --- logging ---                        
                        trainlogger.log_update(
                            train_loss_ema, test_acc, n_train_instances, n_test_instances,
                            elapsed_seconds_from_last_check, batch_size, learning_rates[lr_i], 't' if model_updated else 'f')
                        
                        # --- check for early stopping
                        if checks_with_no_improvement >= early_stopping_checks:
                            if lr_i + 1 < len(learning_rates):
                                lr_i += 1
                                checks_with_no_improvement = 0
                                print("   *** %d checks with no improvements -> using a smaller learning_rate = %f" % (
                                    early_stopping_checks, learning_rates[lr_i]))
                            else:
                                print("   *** %d checks with no improvements -> early stopping :(" % early_stopping_checks)
                                return
                        
                        # --- reset check variables
                        elapsed_seconds_from_last_check = 0
            print('====== TIMEOUT ======')

In [48]:
train_batch_size = 100100
train_minibatches = generate_minibatches(train_instances, train_batch_size)
sanity_check_minibatches(train_minibatches)

n_tuples =  10000276
n_batches =  100


In [49]:
test_batch_size = 101000
test_minibatches = generate_minibatches(test_instances, test_batch_size)
sanity_check_minibatches(test_minibatches)

n_tuples =  402101
n_batches =  4


In [50]:
learning_rates = get_decaying_learning_rates(1e-3, 1e-4, 0.6)
learning_rates

[0.001,
 0.0006,
 0.00035999999999999997,
 0.00021599999999999996,
 0.00012959999999999998]

In [51]:
FINE_GRAINED_THRESHOLD, VISUAL_CONFIDENCE_THRESHOLD

(0.6, 0.1)

In [52]:
# MODEL_PATH = '/mnt/workspace/pamessina_models/ugallery/VBPR/v8(10M-400K,rsnt50,u(200)i(100l+100v),+p-np,+npfavc-nfavc,fg.6,vcf.1)/'
MODEL_PATH = '/mnt/workspace/pamessina_models/ugallery/VBPR/v12(10M-400K,rsnt50,u(200)i(100l+100v),+p-np,fg.6,vcf.1)/'
MODEL_PATH

'/mnt/workspace/pamessina_models/ugallery/VBPR/v12(10M-400K,rsnt50,u(200)i(100l+100v),+p-np,fg.6,vcf.1)/'

In [ ]:
# for _v, _wd in zip((9,10,11), (0.0001, 0.001, 0.01)):
    
#     MODEL_PATH = ('/mnt/workspace/pamessina_models/ugallery/VBPR/v%d(10M-400K,rsnt50,'
#               'u(200)i(100l+100v),+p-np,+npfavc-nfavc,fg.6,vcf.1,wd%f)/') % (_v, _wd)

train_network(
    train_minibatches, test_minibatches,
    len(train_instances), len(test_instances),
    batch_size=train_batch_size,    
    pretrained_embeddings=concat_featmat,
    user_latent_dim=200,
    item_latent_dim=100,
    item_visual_dim=100,
    model_path = MODEL_PATH,
    max_seconds_training=3600*3,
    min_seconds_to_check_improvement=60*3,
    early_stopping_checks=2,
    weight_decay=0,
    learning_rates=learning_rates,
)

learning_rates =  [0.001, 0.0006, 0.00035999999999999997, 0.00021599999999999996, 0.00012959999999999998]
Variables to be trained:
	 <tf.Variable 'user_latent_factors:0' shape=(2919, 200) dtype=float32_ref>
	 <tf.Variable 'item_latent_factors:0' shape=(13297, 100) dtype=float32_ref>
	 <tf.Variable 'item_latent_biases:0' shape=(13297,) dtype=float32_ref>
	 <tf.Variable 'visual_bias:0' shape=(2048,) dtype=float32_ref>
	 <tf.Variable 'trainable_image_embedding/fc1/kernel:0' shape=(2048, 100) dtype=float32_ref>
	 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>
	 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>
	 <tf.Variable 'user_latent_factors/Adam:0' shape=(2919, 200) dtype=float32_ref>
	 <tf.Variable 'user_latent_factors/Adam_1:0' shape=(2919, 200) dtype=float32_ref>
	 <tf.Variable 'item_latent_factors/Adam:0' shape=(13297, 100) dtype=float32_ref>
	 <tf.Variable 'item_latent_factors/Adam_1:0' shape=(13297, 100) dtype=float32_ref>
	 <tf.Variable 'item_latent_biases/Adam